# 1. Libraries

In [1]:
import numpy as np
import pandas as pd
import os

# 2. Functions

In [2]:
def encode_profile(value):
    if pd.isna(value):
        return None
    if value in {0, 1, 2}:
        return value
    if isinstance(value, str):
        value_upper = value.upper()
        if value_upper in {'0', '1', '2'}:
            return int(value_upper)
        elif value_upper == 'STAFF PASS':
            return 0
        elif value_upper == 'TEMP PASS':
            return 1
        elif value_upper == 'VISITOR PASS':
            return 2
    return None


In [3]:
def time_slot(t):
    if pd.to_datetime('07:00:00').time() <= t <= pd.to_datetime('08:29:00').time():
        return '0700 to 0829'
    elif pd.to_datetime('08:30:00').time() <= t <= pd.to_datetime('09:29:00').time():
        return '0830 to 0929'
    else:
        return '0930 to 1030'

# 2. Load Datasets

## a. Site A

In [4]:
# Load datasets
tmp_path = "./" 
files_a = [tmp_path + str(x) for x in os.listdir(tmp_path) if x.startswith('SiteA')] 
tmp_a = [pd.read_csv(tmp_file) for tmp_file in files_a ]
access_a = pd.concat(tmp_a)

In [5]:
access_a.sample(5)

,When,Profile,Dept,CardNum
8,4/5/2020 8:19,1,NaN,1801
638,5/5/2020 7:58,1,NaN,2019183
82,22/5/2020 8:16,1,NaN,27301
1066,22/6/2020 8:09,1,NaN,2019111
322,11/6/2020 9:22,0,Dept 18,216002


## b. Site B

In [6]:
# Load datasets
tmp_path = "./" 
files_b = [tmp_path + str(x) for x in os.listdir(tmp_path) if x.startswith('SiteB')] 
tmp_b = [pd.read_csv(tmp_file) for tmp_file in files_b]
access_b = pd.concat(tmp_b)

In [7]:
access_b.sample(5)

,When,Profile,Dept,CardNum
1063,30/5/2020 9:09,0,Dept 4,3492
908,30/4/2020 8:57,0,Dept 4,254402
601,13/6/2020 7:54,1,NaN,201624
2480,26/6/2020 9:28,0,Dept 16,8833245
305,26/4/2020 8:35,0,Dept 4,118725


# 3. Data Preprocessing

## a. Site A

In [8]:
access_a.info()
# When profile is 1 (Temp Pass) or 2 (Vistor Pass),  dept is na

<class 'pandas.core.frame.DataFrame'>
Index: 12192 entries, 0 to 1728
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   When     12192 non-null  object
 1   Profile  12192 non-null  object
 2   Dept     2925 non-null   object
 3   CardNum  12192 non-null  object
dtypes: object(4)
memory usage: 476.2+ KB


In [9]:
access_a['Profile'].unique() # Inconssitent format and Vistor Pass is not fully encoded as 2

array([2, 1, 0, '1', '0', '2', 'Visitor Pass'], dtype=object)

In [10]:
# Encode Profile
access_a['Profile'] = access_a['Profile'].apply(encode_profile)

In [11]:
non_digits_a = access_a['CardNum'].astype(str).str.contains(r'[^0-9]', regex=True)
access_a[non_digits_a].sample(3) # Need to clean up CardNum field

,When,Profile,Dept,CardNum
203,1/6/2020 10:00,0,Dept 8,#REF!
154,1/6/2020 7:58,0,Dept 11,#REF!
157,1/6/2020 9:50,0,Dept 17,#REF!


In [12]:
# Convert to datetime format
access_a['When'] = pd.to_datetime(access_a['When'], format = '%d/%m/%Y %H:%M')
# Convert to str
access_a['CardNum'] = access_a['CardNum'].astype(str)
# Remove non-digit chars
access_a['CardNum'] = access_a['CardNum'].str.replace(r'[^0-9]', '', regex=True)
# Remove rows where cleaned CardNum is empty
access_a = access_a[access_a['CardNum'].str.strip() != '']
# Fill leading 0s in front to make it 8 characers
access_a['CardNum'] = access_a['CardNum'].apply(lambda x: x.zfill(8))
# Add a field to indicate Site A
access_a['Site'] = 'Site A'

## b. Site B

In [13]:
access_b.info()
# When profile is 1 (Temp Pass) or 2 (Vistor Pass),  dept is na
# There are 5 na values in the CardNum field

<class 'pandas.core.frame.DataFrame'>
Index: 24499 entries, 0 to 2577
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   When     24499 non-null  object
 1   Profile  24499 non-null  object
 2   Dept     14399 non-null  object
 3   CardNum  24494 non-null  object
dtypes: object(4)
memory usage: 957.0+ KB


In [14]:
access_b['Profile'].unique() # Inconssitent format and Temp Pass Vistor Pass is not fully encoded as 2

array([0, 1, 2, '0', '1', '2', 'Temp Pass', 'Staff Pass'], dtype=object)

In [15]:
# Encode Profile
access_b['Profile'] = access_b['Profile'].apply(encode_profile)

In [16]:
# na value in the CardNum
access_b[access_b['CardNum'].isna()] # all rofiles are either 1 (Temp Pass) or 2 (Visitor Pass), reasonable to keep

,When,Profile,Dept,CardNum
2429,26/4/2020 7:31,1,NaN,NaN
2430,24/4/2020 7:34,1,NaN,NaN
2431,22/4/2020 7:27,1,NaN,NaN
2283,3/5/2020 8:32,2,NaN,NaN
2269,8/5/2020 9:42,2,NaN,NaN


In [17]:
non_digits_b = access_b['CardNum'].astype(str).str.contains(r'[^0-9]', regex=True)
access_b[non_digits_b].sample(3) # Need to clean up CardNum col

,When,Profile,Dept,CardNum
2404,23/4/2020 8:32,1,NaN,#REF!
2283,3/5/2020 8:32,2,NaN,NaN
2149,21/4/2020 8:26,1,NaN,#REF!


In [18]:
# Convert to datetime format
access_b['When'] = pd.to_datetime(access_b['When'], format = '%d/%m/%Y %H:%M')
# Convert to str
access_b['CardNum'] = access_b['CardNum'].astype(str)
# Remove non-digit chars
access_b['CardNum'] = access_b['CardNum'].str.replace(r'[^0-9]', '', regex=True)
# Remove rows where cleaned CardNum is empty
access_b = access_b[access_b['CardNum'].str.strip() != '']
# Fill leading 0s in front to make it 8 characers
access_b['CardNum'] = access_b['CardNum'].apply(lambda x: x.zfill(8))
# Add a field to indicate Site B
access_b['Site'] = 'Site B'

In [19]:
access_b.sample(3)

,When,Profile,Dept,CardNum,Site
386,2020-05-15 08:09:00,0,Dept 4,00156410,Site B
1030,2020-05-20 09:35:00,0,Dept 11,00256864,Site B
992,2020-06-23 09:08:00,1,NaN,00201683,Site B


# 4. Combine Site A and Site B Datasets

In [20]:
access_data = pd.concat([access_a, access_b], ignore_index=True)

In [21]:
access_data.head()

,When,Profile,Dept,CardNum,Site
0,2020-04-20 07:17:00,2,NaN,00001001,Site A
1,2020-04-21 07:10:00,2,NaN,00001001,Site A
2,2020-04-22 07:09:00,2,NaN,00001001,Site A
3,2020-04-23 07:16:00,2,NaN,00001001,Site A
4,2020-04-24 07:25:00,2,NaN,00001001,Site A


# 5. Staff_Access

In [22]:
# Extract Staff Profile
staff_access = access_data[access_data['Profile']==0]

In [23]:
staff_access.head()

,When,Profile,Dept,CardNum,Site
72,2020-04-22 08:40:00,0,Dept 5,00021003,Site A
73,2020-04-20 08:30:00,0,Dept 5,00021003,Site A
194,2020-04-20 08:40:00,0,Dept 11,00039802,Site A
197,2020-04-20 07:41:00,0,Dept 5,00042117,Site A
198,2020-04-24 07:35:00,0,Dept 18,00042403,Site A


In [24]:
staff_access['Date'] = staff_access['When'].dt.date
staff_earliest = staff_access.sort_values('When').groupby(['CardNum', 'Date','Site'], as_index=False).first()
staff_earliest['Time'] = staff_earliest['When'].dt.time

C:\Users\Chun Quan\AppData\Local\Temp\ipykernel_23300\2471317276.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff_access['Date'] = staff_access['When'].dt.date


In [25]:
# Create a field to classfiy time
staff_earliest['Slot'] = staff_earliest['Time'].apply(time_slot)

In [26]:
staff_earliest.head()

,CardNum,Date,Site,When,Profile,Dept,Time,Slot
0,00000902,2020-05-25,Site B,2020-05-25 08:09:00,0,Dept 4,08:09:00,0700 to 0829
1,00000902,2020-05-26,Site A,2020-05-26 08:36:00,0,Dept 5,08:36:00,0830 to 0929
2,00000902,2020-05-27,Site A,2020-05-27 08:23:00,0,Dept 5,08:23:00,0700 to 0829
3,00000902,2020-05-27,Site B,2020-05-27 08:08:00,0,Dept 4,08:08:00,0700 to 0829
4,00000902,2020-05-28,Site A,2020-05-28 08:27:00,0,Dept 5,08:27:00,0700 to 0829


# 6. Export to CSV

In [27]:
staff_earliest.to_csv('./staff_earliest.csv', index=False)